In [2]:
import requests
import pandas as pd 
import numpy as np
import os
import csv
import re
import time
import datetime as dt
from bs4 import BeautifulSoup

In [13]:
def get_soup(url:str, delay=0) -> BeautifulSoup:
  """Simple wrapper for getting beautiful soup object from url with sleep delay
  
  Args: 

  url (str): url you're scraping

  delay (int): time you want to wait between next request (default 0)
  """
  result = requests.get(url)
  time.sleep(delay)
  return BeautifulSoup(result.content, "html.parser") 

def get_year_urls(uscrn_directory:str) -> list: 
  """
  Retrieves the URLs for every year's page in the given USCRN directory.
  
  Arguments:
  uscrn_directory (str): Either 'hourly02' or 'subhourly01' (i.e. source of wind data)

  Returns:
  year_urls (list): A list of URLs for every year's page.
  """

  if uscrn_directory not in ("hourly02", 'subhourly01'):
    raise Exception(f"Invalid directory given: {uscrn_directory} -- give 'hourly02' or 'subhourly01'")
  
  url = f"https://www.ncei.noaa.gov/pub/data/uscrn/products/{uscrn_directory}/"
  soup = get_soup(url, 1)

  # Wind data is first available in 2012
  start_year = 2012 if uscrn_directory == "subhourly01" else 2000

  links = soup.find_all("a") 
  years = [str(y).zfill(1) for y in range(start_year, 2024)]
  year_urls = [url + link['href'] for link in links if link['href'].rstrip('/') in years]
  return year_urls

def get_file_urls(uscrn_directory:str) -> list: 
  """
  Retrieves the URLs for every file contained on each year's page in the given USCRN directory

  Arguments:
  uscrn_directory (str): Either 'hourly02' or 'subhourly01'

  Returns: 
  file_urls (list): A list of file URLs.
  """

  if uscrn_directory not in ("hourly02", 'subhourly01'):
    raise Exception(f"Invalid directory given: {uscrn_directory} -- give 'hourly02' or 'subhourly01'")

  year_urls = get_year_urls(uscrn_directory)

  file_urls = []
  for url in year_urls: 
    soup = get_soup(url)
    file_links = soup.find_all('a', href=re.compile(r'AK.*\.txt'))
    if file_links:
      new_file_urls = [url + link.getText() for link in file_links]
      file_urls.extend(new_file_urls)
  return file_urls

def get_station_location(url) -> str: 
  """
  Extracts the name of the station from a given URL.
  
  Args:
  url (str): The URL to extract the station name from.
  
  Returns:
  station_location (str): The name of the station.
  """
  regex = r"([St.]*[A-Z][a-z]+_*[A-Za-z]*).*.txt" 
  file_name = re.search(regex, url).group(0)
  station_location = re.sub("(_formerly_Barrow.*|_[0-9].*)", "", file_name)
  return  station_location

In [12]:
file_urls = ['https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2006/CRNS0101-05-2006-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2006/CRNS0101-05-2006-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2006/CRNS0101-05-2006-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2006/CRNS0101-05-2006-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2007/CRNS0101-05-2007-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2007/CRNS0101-05-2007-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2007/CRNS0101-05-2007-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2007/CRNS0101-05-2007-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2008/CRNS0101-05-2008-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2008/CRNS0101-05-2008-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2008/CRNS0101-05-2008-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2008/CRNS0101-05-2008-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2009/CRNS0101-05-2009-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2009/CRNS0101-05-2009-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2009/CRNS0101-05-2009-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2009/CRNS0101-05-2009-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2009/CRNS0101-05-2009-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2009/CRNS0101-05-2009-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2010/CRNS0101-05-2010-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2010/CRNS0101-05-2010-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2010/CRNS0101-05-2010-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2010/CRNS0101-05-2010-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2010/CRNS0101-05-2010-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2010/CRNS0101-05-2010-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2010/CRNS0101-05-2010-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2010/CRNS0101-05-2010-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2011/CRNS0101-05-2011-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2011/CRNS0101-05-2011-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2011/CRNS0101-05-2011-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2011/CRNS0101-05-2011-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2011/CRNS0101-05-2011-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2011/CRNS0101-05-2011-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2011/CRNS0101-05-2011-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2011/CRNS0101-05-2011-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2011/CRNS0101-05-2011-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2012/CRNS0101-05-2012-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2012/CRNS0101-05-2012-AK_Gustavus_2_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2012/CRNS0101-05-2012-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2012/CRNS0101-05-2012-AK_King_Salmon_42_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2012/CRNS0101-05-2012-AK_Metlakatla_6_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2012/CRNS0101-05-2012-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2012/CRNS0101-05-2012-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2012/CRNS0101-05-2012-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2012/CRNS0101-05-2012-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2012/CRNS0101-05-2012-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2012/CRNS0101-05-2012-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2012/CRNS0101-05-2012-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_Glennallen_64_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_Gustavus_2_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_King_Salmon_42_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_Metlakatla_6_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Deadhorse_3_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Glennallen_64_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Gustavus_2_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Ivotuk_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_King_Salmon_42_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Metlakatla_6_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Ruby_44_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Deadhorse_3_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Denali_27_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Glennallen_64_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Gustavus_2_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Ivotuk_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_King_Salmon_42_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Metlakatla_6_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Ruby_44_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Selawik_28_E.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Deadhorse_3_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Denali_27_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Glennallen_64_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Gustavus_2_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Ivotuk_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_King_Salmon_42_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Metlakatla_6_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Ruby_44_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Selawik_28_E.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Yakutat_3_SSE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Cordova_14_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Deadhorse_3_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Denali_27_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Glennallen_64_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Gustavus_2_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Ivotuk_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_King_Salmon_42_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Metlakatla_6_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Ruby_44_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Selawik_28_E.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Toolik_Lake_5_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Yakutat_3_SSE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Bethel_87_WNW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Cordova_14_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Deadhorse_3_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Denali_27_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Glennallen_64_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Gustavus_2_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Ivotuk_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_King_Salmon_42_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Metlakatla_6_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Ruby_44_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Selawik_28_E.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Toolik_Lake_5_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Yakutat_3_SSE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Aleknagik_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Bethel_87_WNW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Cordova_14_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Deadhorse_3_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Denali_27_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Glennallen_64_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Gustavus_2_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Ivotuk_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_King_Salmon_42_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Metlakatla_6_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Ruby_44_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Selawik_28_E.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Toolik_Lake_5_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Yakutat_3_SSE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Aleknagik_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Bethel_87_WNW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Cordova_14_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Deadhorse_3_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Denali_27_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Glennallen_64_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Gustavus_2_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Ivotuk_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_King_Salmon_42_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Metlakatla_6_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Ruby_44_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Selawik_28_E.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Toolik_Lake_5_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Yakutat_3_SSE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Aleknagik_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Bethel_87_WNW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Cordova_14_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Deadhorse_3_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Denali_27_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Glennallen_64_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Gustavus_2_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Ivotuk_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_King_Salmon_42_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Metlakatla_6_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Ruby_44_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Selawik_28_E.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Toolik_Lake_5_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Yakutat_3_SSE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Aleknagik_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Bethel_87_WNW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Cordova_14_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Deadhorse_3_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Denali_27_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Glennallen_64_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Gustavus_2_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Ivotuk_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_King_Salmon_42_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Metlakatla_6_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Ruby_44_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Selawik_28_E.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Toolik_Lake_5_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Yakutat_3_SSE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Aleknagik_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Bethel_87_WNW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Cordova_14_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Deadhorse_3_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Denali_27_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Glennallen_64_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Gustavus_2_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Ivotuk_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_King_Salmon_42_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Metlakatla_6_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Ruby_44_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Selawik_28_E.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Toolik_Lake_5_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Yakutat_3_SSE.txt']

In [21]:
def get_raw_rows(file_urls, output_file) -> None:
  """
   file.

  Args:
    file_urls (list): A list of URLs where weather station data can be found.
    output_file (str): The path to the output CSV file.

  Returns:
    None
  """
  for url in file_urls:
    # Get location from url
    station_location = get_station_location(url)
    # Get new rows 
    soup = get_soup(url, delay=.5)
    lines = [re.split('\s+', line) for line in str(soup).strip().splitlines()]
    # We're only scraping this data for the wind information, so we ignore rows that don't have any (i.e wind < 0)
    wind_cols = [[station_location] + line[:5] + line[-2:] for line in lines if float(line[-2]) >= 0]
    # Write rows to CSV
    if wind_cols:
      mode = "a" if os.path.isfile(output_file) else "w"
      with open(output_file, f"{mode}") as f:
        writer = csv.writer(f)
        writer.writerows(wind_cols)
      del wind_cols

In [22]:
get_raw_rows(file_urls=file_urls)

[['Fairbanks', '26494', '20120809', '1705', '20120809', '0805', '0.00', '3'], ['Fairbanks', '26494', '20120809', '1710', '20120809', '0810', '0.00', '3'], ['Fairbanks', '26494', '20120809', '1715', '20120809', '0815', '0.00', '3'], ['Fairbanks', '26494', '20120809', '1720', '20120809', '0820', '0.00', '3']]
